# Download yolo 

In [1]:
!git clone https://github.com/ultralytics/yolov5  # clone repo
%cd yolov5
%pip install -qr requirements.txt  # install dependencies
# %pip install opencv-python

import torch
from IPython.display import Image, clear_output  # to display images
import os

clear_output()
print(f"Working directory: {os.getcwd()}")
print(f"Setup complete. Using torch {torch.__version__} ({torch.cuda.get_device_properties(0).name if torch.cuda.is_available() else 'CPU'})")

Working directory: /home/joshdw/Documents/src/FlockSentry/training/yolov5
Setup complete. Using torch 1.10.0+cu102 (NVIDIA GeForce GTX 1650 Ti with Max-Q Design)


# Download our training data

In [2]:
input("Download training data from drive and unpack into `all_image_data`.  Press ENTER when done.")
CUSTOM_DATA_PATH = '../all_image_data/'
print(f"all_image_data file count: {len(os.listdir(CUSTOM_DATA_PATH))}")
COCO_DATA_PATH = "../datasets/coco/"
COCO_LABELS_PATH = COCO_DATA_PATH + "/labels/val2017/"
COCO_IMAGES_PATH = COCO_DATA_PATH + "/images/val2017/"
YOLO_DATA_PATH = "./data/"

all_image_data file count: 623


# Download coco data

In [3]:
download_coco = True
if os.path.exists(COCO_DATA_PATH):
    response = input(f"Replace coco dataset? [Y/n]")
    download_coco = response.lower() in ["y", "yes"]

if download_coco:
    # Download COCO val
    torch.hub.download_url_to_file('https://ultralytics.com/assets/coco2017val.zip', 'tmp.zip')
    !unzip -oq tmp.zip -d ../datasets && rm tmp.zip

In [4]:
#person, bird, cat, dog, horse, sheep, cow, bear
keep = ["0", "14", "15", "16", "17", "18", "19", "21"]
replace_dict = {"0": "4", "14": "3", "15": "5", "16": "6", "17": "7", "18": "8", "19": "9", "21": "10"}

for filename in os.listdir(COCO_LABELS_PATH):
  #lines we want to keep
  useful_lines = []
  rIdx = 0
  wIdx = 0 
  #read all the lines in the file
  with open(COCO_LABELS_PATH + filename, "r") as f:
    lines = f.readlines()
    for line in lines:
      items = line.split(" ") #items[0] = class
      if(items[0] in keep):
        useful_lines.append(rIdx) #note that line (all other lines will be removed)
      rIdx = rIdx + 1
  #copy only the lines we want to keep back to the file 
  with open(COCO_LABELS_PATH + filename, "w") as f:
    for line in lines:
        if wIdx in useful_lines:
            items = line.split(" ") #items[0] = class
            items[0] = replace_dict[items[0]]
            new_line = " ".join(items)
            f.write(new_line)
        wIdx = wIdx + 1

In [5]:
#remove empty text files and the images that are associated with them
for filename in os.listdir(COCO_LABELS_PATH):
    label_path = COCO_LABELS_PATH + filename
    if(os.path.getsize(label_path) == 0):
      os.remove(label_path)
      image_path = COCO_DATA_PATH + "images/val2017/" + filename.replace(".txt", ".jpg")
      os.remove(image_path)

# Remove duplicate images from our training data

It is possible that two team members downloaded and annotated the same image from the internet.  This should clean that up.

In [6]:
import cv2
import os
import numpy as np
from matplotlib import pyplot as plt


duplicate_images = []

# Gather filenames of all images
img_fnames = [fname for fname in os.listdir(CUSTOM_DATA_PATH)
    if fname.endswith(".jpg") or fname.endswith(".png") or fname.endswith(".jpeg")]

chicken_images = []
i = 0
for img_fname in img_fnames:  # Open, normalize, and append all images
    img = cv2.resize(cv2.imread(CUSTOM_DATA_PATH + img_fname), (640, 480))
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    chicken_images.append(img)

In [7]:
# Find images that look like the exact same image and nominate them for deletion
for i, img in enumerate(chicken_images):
    for j, other_img in enumerate(chicken_images[i+1:]):
        # Test similarity with root means squared
        diff = img - other_img
        distance = np.sqrt(np.sum(np.square(diff)))
        if distance < 100:
            # Display both images
            plt.imshow(img)
            plt.show()
            plt.imshow(other_img)
            plt.show()
            other_fname = img_fnames[i+1+j]
            # Ask if the second image should be deleted ...
            print(f"Duplicate images: {img_fnames[i]} and {other_fname}")
            should_delete = input(f"Delete {other_fname}? [Y/n]")
            # ... and delete the second image if yes
            if should_delete in ["y", "Y", ""]:
                print(f"Deleting {other_fname} and matching label file...")
                os.remove(CUSTOM_DATA_PATH + "".join(other_fname.split(".")[:-1]) + ".txt")
                os.remove(CUSTOM_DATA_PATH + other_fname)

# Allocate a certain amount for each category
#### to not flood the chicken images 

In [8]:
"""
Moves a random sample of one tenth of images and labels to the validation data
folder, and the rest to the training data folder.
"""

import shutil
from glob import glob
import os
import random

# Tuneable parameters for data amounts
VALIDATION_PROPORTION = 0.2
COCO_PROPORTION = 0.0

# Find all of the custom data label files
custom_label_fnames = glob(CUSTOM_DATA_PATH + "*.txt")
n = int(len(custom_label_fnames) * VALIDATION_PROPORTION)
custom_validation_labels = random.sample(custom_label_fnames, n)
print(f"Custom images: {n} val, {len(custom_label_fnames) - n} train")

# Find all of the coco data label files
coco_label_fnames = glob(COCO_DATA_PATH + "labels/val2017/*.txt")
n = int(len(custom_label_fnames)*COCO_PROPORTION/(1-COCO_PROPORTION))
coco_label_fnames = random.sample(coco_label_fnames, n)
n = int(len(coco_label_fnames) * VALIDATION_PROPORTION)
coco_validation_labels = random.sample(coco_label_fnames, n)
print(f"Coco images: {n} val, {len(coco_label_fnames) - n} train")

# Clear the yolov5/data/images and yolov5/data/labels folders
try:
    shutil.rmtree(YOLO_DATA_PATH + "images/")
    shutil.rmtree(YOLO_DATA_PATH + "labels/")
except FileNotFoundError as e:  # We have already deleted the folders
    print(f"Not deleting nonexistent images/ and labels/ folders in {YOLO_DATA_PATH}.")
os.mkdir(YOLO_DATA_PATH + "images/")
os.mkdir(YOLO_DATA_PATH + "labels/")
os.mkdir(YOLO_DATA_PATH + "images/val")
os.mkdir(YOLO_DATA_PATH + "images/train")
os.mkdir(YOLO_DATA_PATH + "labels/val")
os.mkdir(YOLO_DATA_PATH + "labels/train")

def get_matching_img_fname(label_fname):
    """Takes a .txt label filename and returns its matching image filename"""
    label_fname = label_fname.replace('coco/labels/val2017/', 'coco/images/val2017/')
    img_fname = ""
    label_fname_root = label_fname.replace(".txt", "")
    if os.path.exists(f"{label_fname_root}.jpg"):
        img_fname = f"{label_fname_root}.jpg"
    elif os.path.exists(f"{label_fname_root}.jpeg"):
        img_fname = f"{label_fname_root}.jpeg"
    elif os.path.exists(f"{label_fname_root}.png"):
        img_fname = f"{label_fname_root}.png"
    else:
        print(f"No matching .jpg or .png file for {label_fname}.  Continuing...")
        return None
    return img_fname

# TODO: resume commenting here
for label_fname in custom_label_fnames:
    img_fname = get_matching_img_fname(label_fname)
    if not img_fname:
        continue

    if label_fname in custom_validation_labels:
        shutil.copy2(img_fname, YOLO_DATA_PATH + "images/val/")
        shutil.copy2(label_fname, YOLO_DATA_PATH + "labels/val/")
    else:
        shutil.copy2(img_fname, YOLO_DATA_PATH + "images/train/")
        shutil.copy2(label_fname, YOLO_DATA_PATH + "labels/train/")

for label_fname in coco_label_fnames:
    img_fname = get_matching_img_fname(label_fname)
    if not img_fname:
        continue

    if label_fname in coco_validation_labels:
        shutil.copy2(img_fname, YOLO_DATA_PATH + "images/val/")
        shutil.copy2(label_fname, YOLO_DATA_PATH + "labels/val/")
    else:
        shutil.copy2(img_fname, YOLO_DATA_PATH + "images/train/")
        shutil.copy2(label_fname, YOLO_DATA_PATH + "labels/train/")

Custom images: 62 val, 250 train
Coco images: 0 val, 0 train
No matching .jpg or .png file for ../all_image_data/labels.txt.  Continuing...


# Train the model with our custom dataset

In [11]:
!python3 train.py --img 640 --batch 12 --epochs 500 --data ../custom.yaml --cfg models/yolov5n.yaml --weights yolov5s.pt --cache

train: weights=yolov5s.pt, cfg=models/yolov5n.yaml, data=../custom.yaml, hyp=data/hyps/hyp.scratch.yaml, epochs=500, batch_size=12, imgsz=640, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, evolve=None, bucket=, cache=ram, image_weights=False, device=, multi_scale=False, single_cls=False, adam=False, sync_bn=False, workers=8, project=runs/train, name=exp, exist_ok=False, quad=False, linear_lr=False, label_smoothing=0.0, patience=100, freeze=0, save_period=-1, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest
github: ⚠️ YOLOv5 is out of date by 264 commits. Use `git pull` or `git clone https://github.com/ultralytics/yolov5` to update.
YOLOv5 🚀 v6.0-94-g47fac9f torch 1.10.0+cu102 CUDA:0 (NVIDIA GeForce GTX 1650 Ti with Max-Q Design, 3912MiB)

hyperparameters: lr0=0.01, lrf=0.1, momentum=0.937, weight_decay=0.0005, warmup_epochs=3.0, warmup_momentum=0.8, warmup_bias_lr=0.1, box=0.05, cls=0.5, cls_pw=1.0, obj=1.0, obj_pw=1.0

# Run model on something
#### probably a video with chickens

**NOTE:** You will need to modify `exp` to the latest exp folder generated from the training command above.

In [ ]:
!python3 detect.py --weights runs/train/<LATEST EXP>/weights/best.pt --img 640 --conf 0.3 --source ../test.mp4

detect: weights=['runs/train/exp3/weights/best.pt'], source=/home/joshdw/Downloads/home/joshdw/Downloads/yt_download_dMW80rrTDb4.mp4, imgsz=[640, 640], conf_thres=0.3, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=False, save_conf=False, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=runs/detect, name=exp, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False
YOLOv5 🚀 v6.0-94-g47fac9f torch 1.10.0+cu102 CUDA:0 (NVIDIA GeForce GTX 1650 Ti with Max-Q Design, 3912MiB)

Fusing layers... 
Model Summary: 213 layers, 7050580 parameters, 0 gradients, 15.9 GFLOPs
Traceback (most recent call last):
  File "/home/joshdw/Documents/src/FlockSentry/training/yolov5/detect.py", line 244, in <module>
    main(opt)
  File "/home/joshdw/Documents/src/FlockSentry/training/yolov5/detect.py", line 239, in main
    run(**vars(opt))
  File "/home/joshdw/.local/lib/python3.9/site-pack